## CPU-bound в async

In [2]:
import asyncio

In [ ]:
async def foo():
    print('🚀 Running foo...')
    for i in range(7_000):          ## CPU-bound task блокирует поток!
        i**i
        if i % 100 == 0:            # Раз в 100 итераций
            await asyncio.sleep(0)  # Переключаем контекст
    print('✅ foo: OK')
    return 'foo'

async def bar():
    print('🚀 Running bar...')
    await asyncio.sleep(1)
    print('✅ bar: OK')
    return 'bar'

res = await asyncio.gather(foo(), bar())

print(res)

🚀 Running foo...
🚀 Running bar...
✅ bar: OK
✅ foo: OK counts 70
['foo', 'bar']
